In [9]:
import Tools, healpy
reload(Tools)
import Analysis
A = Analysis.Analysis()
self = A
import sys

import multiprocessing as mp

def AddGalpropTemplate(basedir='/data/fermi_diffuse_models/galprop.stanford.edu/PaperIISuppMaterial/OUTPUT',
               tag='SNR_z4kpc_R20kpc_Ts150K_EBV2mag', verbosity=1, multiplier=1., bremsfrac=None, E_subsample=3):
    """
    This method takes a base analysis prefix, along with an X_CO profile and generates the combined diffuse template, 
    or components of the diffuse template. 
    
    :param basedir: Base directory to read from
    :param tag: Tag for the galprop file.  This is the part between '_54_' and '.gz'. 
    :param verbosity: 0 is quiet, >1 prints status.
    :param multiplier: Blur each map using Gaussian kernel with sigma=FWHM_PSF*multiplier/2
    :param bremsfrac: If None, brems is treated as independent.  Otherwise Brem normalization
        is linked to Pi0 normalization, scaled by a factor bremsfrac.
    :param E_subsample: Number of energy sub bins to use when integrating over each energy band.
    """
    
    #---------------------------------------------------------------------------------
    # Load templates
    
    if verbosity>0:
        print 'Loading FITS'
        
    comps, comps_new = {}, {}
    comps['ics'] = pyfits.open(basedir+'/ics_isotropic_healpix_54_'+tag+'.gz')[1].data.field(0).T
    comps['pi0'] = pyfits.open(basedir+'/pi0_decay_healpix_54_'+tag+'.gz')[1].data.field(0).T
    comps['brem'] = pyfits.open(basedir+'/bremss_healpix_54_'+tag+'.gz')[1].data.field(0).T
    
    energies = pyfits.open(basedir+'/bremss_healpix_54_'+tag+'.gz')[2].data.field(0)
    nside_in = np.sqrt(comps['ics'].shape[1]/12)
    
    # Init new templates
    comps_new['ics'] = np.zeros((self.n_bins, 12*self.nside**2))
    comps_new['pi0'] = np.zeros((self.n_bins, 12*self.nside**2))
    comps_new['brem'] = np.zeros((self.n_bins, 12*self.nside**2))
    
    #---------------------------------------------------------------------------------
    # Now we integrate each model over the energy bins...
    # 
    # Multiprocessing for speed. There is an async callback which applies each result to 
    # the arrays.  Not sure why RunAsync needs new thread pool for each component, but this
    # works and decreases memory footprint. 
    def callback(result):
        idx, comp, dat = result 
        comps_new[comp][idx] = dat
    
    def RunAsync(component):
        p = mp.Pool(mp.cpu_count())
        for i_E in range(self.n_bins):    
            p.apply_async(Tools.__AsyncInterpolateHealpix, 
                           [comps[component], energies, self.bin_edges[i_E], self.bin_edges[i_E+1], i_E,
                            component, 3, self.nside],
                           callback=callback)
        p.close()
        p.join()
    
    # For each component, run the async sampling/sizing. 
    for key in comps:
        if verbosity>0:
            print 'Integrating and Resampling', key, 'templates...'
        RunAsync(key)
        
        
    #---------------------------------------------------------------------------------
    # Now we just need to add the templates to the active template stack
    
    # Delete previous keys for diffuse model
    for key in ['Brems', 'Pi0', 'ICS', 'FermiDiffuse', 'Pi0+Brems']:
        self.templateList.pop(key, None)
    
    
    self.AddTemplate(name='ICS', healpixCube=comps_new['ics'], fixSpectrum=True, fixNorm=False,
                       value=1, ApplyIRF=True, sourceClass='GEN', limits=[0, 10.], multiplier=multiplier)
    
    if bremsfrac is None:
        self.AddTemplate(name='Brems', healpixCube=comps_new['brem'], fixSpectrum=True, fixNorm=False,
                           value=1, ApplyIRF=True, sourceClass='GEN', limits=[0, 10.], multiplier=multiplier)
        self.AddTemplate(name='Pi0', healpixCube=comps_new['pi0'], fixSpectrum=True, fixNorm=False,
                           value=1, ApplyIRF=True, sourceClass='GEN', limits=[0, 10.], multiplier=multiplier)
    
    else: 
        self.AddTemplate(name='Pi0+Brems', healpixCube=comps_new['pi0']+bremsfrac*comps_new['brem'],
                           fixSpectrum=True, fixNorm=False,
                           value=1, ApplyIRF=True, sourceClass='GEN', limits=[0, 10.], multiplier=multiplier)


    
%time AddGalpropTemplate()



Loading FITS
Integrating and Resampling ics templates...
Integrating and Resampling pi0 templates...
Integrating and Resampling brem templates...
CPU times: user 3min 23s, sys: 4.15 s, total: 3min 28s
Wall time: 40.5 s


In [17]:
results = []

def test(x):
    return x*x

def callback(result):
    results.append(result)
    
p = mp.Pool(mp.cpu_count())    

for i in range(10):
    p.apply_async(test, [i,], callback=callback)
p.close()
p.join()

print results

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
